In [96]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, adjusted_rand_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

In [97]:
df = pd.read_csv("merged.csv")
df = df.drop(['Unnamed: 0'], axis=1)
df

,part_id,fried,gender,age,hospitalization_one_year,hospitalization_three_years,ortho_hypotension,vision,audition,weight_loss,...,BathRoom,BedRoom,Entrance,Garage,Hall,Kitchen,LivingRoom,Office,Outdoor,Storage
0,1001,0,1,75,0,0.0,0,1,2,0.0,...,7.430730,32.304786,0.0,0.0,47.355164,0.000000,6.202771,0.000000,6.706549,0.0
1,1003,1,0,72,0,0.0,0,1,1,0.0,...,0.273620,0.000000,0.0,0.0,0.177048,77.804603,0.000000,0.000000,21.744729,0.0
2,1005,1,1,83,0,4.0,1,1,0,0.0,...,0.000000,0.000000,0.0,0.0,99.777667,0.000000,0.222333,0.000000,0.000000,0.0
3,1006,0,0,72,0,1.0,0,1,2,0.0,...,98.305596,0.683120,0.0,0.0,0.304725,0.391789,0.000000,0.000000,0.314771,0.0
4,1007,0,1,70,0,0.0,0,1,2,0.0,...,3.496115,71.624121,0.0,0.0,1.775805,0.000000,0.147984,0.000000,22.955975,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,3593,1,0,75,0,1.0,1,2,2,0.0,...,12.498823,0.847697,0.0,0.0,0.000000,8.985589,0.000000,76.678911,0.988980,0.0
269,3594,0,0,71,0,0.0,0,2,2,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,99.975994,0.001847,0.001154,0.021005,0.0
270,3600,0,1,77,0,0.0,0,2,2,0.0,...,12.019929,3.923604,0.0,0.0,0.000000,0.000000,79.468549,0.000000,4.587918,0.0
271,3601,1,0,84,0,0.0,1,2,1,0.0,...,0.000000,0.014536,0.0,0.0,0.000000,0.000000,90.734180,0.007477,9.243806,0.0


In [98]:

true_label_names = list(df.columns)
true_label_names = np.array(true_label_names)

data = df.to_numpy()

In [99]:
data

array([[1.00100000e+03, 0.00000000e+00, 1.00000000e+00, ...,
        0.00000000e+00, 6.70654912e+00, 0.00000000e+00],
       [1.00300000e+03, 1.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 2.17447288e+01, 0.00000000e+00],
       [1.00500000e+03, 1.00000000e+00, 1.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [3.60000000e+03, 0.00000000e+00, 1.00000000e+00, ...,
        0.00000000e+00, 4.58791779e+00, 0.00000000e+00],
       [3.60100000e+03, 1.00000000e+00, 0.00000000e+00, ...,
        7.47728448e-03, 9.24380601e+00, 0.00000000e+00],
       [3.61100000e+03, 0.00000000e+00, 1.00000000e+00, ...,
        0.00000000e+00, 4.99546691e+01, 0.00000000e+00]])

In [100]:
label_encoder = LabelEncoder()

true_labels = label_encoder.fit_transform(true_label_names)

true_labels[:5]

array([47, 24, 29, 11, 33])

In [101]:
label_encoder.classes_

n_clusters = len(label_encoder.classes_)

In [102]:
preprocessor = Pipeline(
    [
        ("scaler", MinMaxScaler()),
        ("pca", PCA(n_components=2, random_state=42)),
    ]
)

In [103]:
clusterer = Pipeline(
   [
       (
           "kmeans",
           KMeans(
               n_clusters=n_clusters,
               init="k-means++",
               n_init=50,
               max_iter=500,
               random_state=42,
           ),
       ),
   ]
)

In [104]:
pipe = Pipeline(
    [
        ("preprocessor", preprocessor),
        ("clusterer", clusterer)
    ]
)

In [105]:
pipe.fit(data)

Pipeline(steps=[('preprocessor',
                 Pipeline(steps=[('scaler', MinMaxScaler()),
                                 ('pca',
                                  PCA(n_components=2, random_state=42))])),
                ('clusterer',
                 Pipeline(steps=[('kmeans',
                                  KMeans(max_iter=500, n_clusters=60, n_init=50,
                                         random_state=42))]))])

In [106]:
preprocessed_data = pipe["preprocessor"].transform(data)

predicted_labels = pipe["clusterer"]["kmeans"].labels_

silhouette_score(preprocessed_data, predicted_labels)
predicted_labels.shape

(273,)

In [107]:
adjusted_rand_score(true_labels, predicted_labels)

ValueError: Found input variables with inconsistent numbers of samples: [60, 273]